In this algorithm we take the last one a bit further by now allowing for short selling via defining a weight function.  This improves diversification greatly.  It's definitely complicated to make a good weight function since it's heavily dependent on the stocks and our data pipeline, but this serves the purpose well enough.

Still we don't have a lot of short selling taking place.  In this case we'll want to make a data pipeline to take a lot of assets and pricing data (say the whole S&P 500) and shrink it into maybe 30-50 stocks that we'll trade, ensuring to have specific long/short signals so that we're not too long heavy.  We'll look more into pipelining in our next algo.  

In [ ]:
import pandas as pd
result = pd.read_html("http://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
data = result[0]
companies = list(data[0][1:])

def make_symbols(symbols: list, length=-1):
    new_list = []
    for asset in symbols:
        try:
            new_list.append(symbol(asset))
        except SymbolNotFound:
            pass
    shuffle(new_list)
    if length == -1:
        return new_list
    return new_list[:length]

In [ ]:
from zipline.api import order_target_percent, symbol, order_target, set_max_leverage
from zipline import run_algorithm
import warnings
import pyfolio as pf
from random import shuffle
from ta.momentum import StochRSIIndicator
from zipline.errors import SymbolNotFound, CannotOrderDelistedAsset

warnings.filterwarnings("ignore")


STOCKS = companies
MAX_BAR_HOLD = 3



def initialize(algo):
    algo.stocks = STOCKS
    algo.symbol_ids = make_symbols(STOCKS, 10)
    print(algo.symbol_ids)
    algo.has_open_positions = {symbol: False for symbol in algo.symbol_ids} 
    algo.bars_held = {symbol: 0 for symbol in algo.symbol_ids} # number of bars (ie timesteps) we've held an asset

    
def purchase_share(algo, data):
    recent = data.history(assets=algo.symbol_ids, fields='close', bar_count=60, frequency='1d')
    short_mean = recent[-10:].mean()
    long_mean = recent[-50:].mean()
    return (short_mean > long_mean) # short term is outperforming long term

def make_weights(algo, data):
    recent = data.history(assets=algo.symbol_ids, fields='close', bar_count=20, frequency='1d')
    short_mean = recent[-5:].mean()
    long_mean = recent[-15:].mean()
    # simple percent change of two means.  Also accounts for short selling
    weights = (short_mean - long_mean) / long_mean 
    normalized_weights = weights / abs(weights).sum()
    return normalized_weights


def max_purchase_weight(security, weights):
    max_allowed_weight = len(STOCKS) / 100.0
    if weights[security] < 0:
        return max(-max_allowed_weight, weights[security])
    return min(max_allowed_weight, weights[security])

def handle_data(algo, data):
    purchase = purchase_share(algo, data)
    weights = make_weights(algo, data)
    recent = data.history(assets=algo.symbol_ids, fields='close', bar_count=30, frequency='1d')
    for security in algo.symbol_ids:
        rsi = StochRSIIndicator(recent[security])
        print(rsi.stochrsi())
        if not algo.has_open_positions[security] and algo.bars_held[security] <= MAX_BAR_HOLD and purchase[security]:
            try: 
                order_target_percent(security, max_purchase_weight(security, weights)) 
                algo.has_open_positions[security] = True;
                algo.bars_held[security] += 1;
            except CannotOrderDelistedAsset:
                pass
        elif algo.bars_held[security] > MAX_BAR_HOLD or not purchase[security]:
            try:
                order_target_percent(security, 0) # order nothing (sell share)
                algo.has_open_positions[security] = False # reset value for security's open positions
                algo.bars_held[security] = 0
            except CannotOrderDelistedAsset:
                pass
            


start = pd.Timestamp('2015-1-1', tz='utc')
end = pd.Timestamp('2018-1-1', tz='utc')
results = run_algorithm(start=start, end=end, capital_base=1000, initialize=initialize, handle_data=handle_data, 
                        data_frequency='daily')

returns, positions, orders = pf.utils.extract_rets_pos_txn_from_zipline(results)